# PART 1

In [22]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [23]:
URL= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page= requests.get(URL).text

soup= BeautifulSoup(page, "xml")
table=soup.find("table")

print("page scrapped")

page scrapped


In [24]:
column_names = ['PostalCode', 'Borough', 'Neighborhood']

df = pd.DataFrame(columns = column_names)
df

,PostalCode,Borough,Neighborhood


In [25]:
# search all the postcode, borough, and neighborhood
for tr in table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)]=row_data

In [26]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Data preprocessing and cleaning

In [27]:
# Dropping the rows where Borough is 'Not assigned'
df1 =df[df['Borough']!='Not assigned']

# Combining the neighbourhoods with same postalcode
df2 = df1.groupby(['PostalCode', 'Borough'], sort=False).agg(','.join)
df2.reset_index(inplace=True)

# Replacing the name of the neighborhoods which are 'Not assigned' with names of borough
df2['Neighborhood']=np.where(df2['Neighborhood'] == 'Not assigned', df2['Borough'], df2['Neighborhood'])

df2

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [8]:
# Shape of the data frame
df2.shape

(103, 3)

# PART 2

### Importing the csv file containing the latitudes and longitudes for various neighborhoods in canada

In [28]:
lat_long= pd.read_csv('https://cocl.us/Geospatial_data')
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### We now merge the two tables to get the neighborhood's Latitudes and Longitudes

In [29]:
lat_long.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df3=pd.merge(df2, lat_long, on='PostalCode')
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# PART 3

### This part presents the clustering and the plotting of the neighborhoods of Canada  which contains Toronto in their Borough.

In [30]:
import random
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries imported.')

Libraries imported.


In [31]:
df4 = df3[df3['Borough'].str.contains('Toronto', regex=False)]
df4.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


### Visualizing all the neighborhoods of the above data frame

In [32]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### NOTE: The map might not be visible on Github.

### Using KMeans clustering for the clustering of the Neighborhoods

In [33]:
k=5
toronto_cluster=df4.drop(['PostalCode', 'Borough', 'Neighborhood'], 1)
kmeans = KMeans(n_clusters =k, random_state=0).fit(toronto_cluster)
kmeans.labels_
df4.insert(0, 'Cluster Labels',kmeans.labels_)


In [34]:
df4.head()

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,1,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [35]:
# create map
map_clusters = folium.Map(location=[43.651070, -79.347015], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighborhood, cluster in zip(df4['Latitude'], df4['Longitude'], df4['Neighborhood'], df4['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### NOTE: The map might not be visible on Github.